# Exploratory Analysis of FDIC & FFIEC Data

Este cuaderno guía el proceso para unir la lista de bancos quebrados de la FDIC con los reportes financieros del FFIEC. Cada sección agrega contexto en texto antes de ejecutar el código para dejar claro cuál es el objetivo de la celda siguiente.


## 1. Configuración inicial

Importamos las librerías que usaremos durante todo el análisis, configuramos estilos de visualización y ajustamos las opciones de pandas para inspeccionar tablas grandes sin perder columnas importantes.


In [ ]:
# Configuración global de librerías y estilo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import csv
import itertools
from pathlib import Path

# Ajustes de visualización y supresión de advertencias
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")


## 2. Datos FDIC: lista de bancos quebrados

Comenzamos leyendo `download-data.csv`, normalizando encabezados y tipos de datos (por ejemplo, fechas y el certificado `Cert`). También agregamos columnas derivadas como año y trimestre para facilitar el análisis temporal posterior.


In [ ]:
# Cargar y preparar la lista de bancos quebrados de la FDIC
fdic_path = Path('download-data.csv')
if not fdic_path.exists():
    raise FileNotFoundError('No se encontró download-data.csv. Descarga primero la lista de bancos quebrados de la FDIC.')

# Lectura del CSV manejando caracteres especiales en encabezados
fdic_failed = pd.read_csv(fdic_path, encoding='latin-1')
fdic_failed.columns = fdic_failed.columns.str.strip().str.replace(' ', '').str.replace(' ', '_')

# Normalizar identificadores y fechas clave
fdic_failed['Cert'] = fdic_failed['Cert'].astype(str).str.strip()
fdic_failed['Closing_Date'] = pd.to_datetime(fdic_failed['Closing_Date'], errors='coerce')
fdic_failed['year'] = fdic_failed['Closing_Date'].dt.year
fdic_failed['quarter'] = fdic_failed['Closing_Date'].dt.quarter

print(f'Registros FDIC: {len(fdic_failed):,}')
print(f'Rango temporal: {fdic_failed["Closing_Date"].min()} -> {fdic_failed["Closing_Date"].max()}')
fdic_failed.head()


In [ ]:
# Resumen estructural y estadístico del dataset FDIC
print(fdic_failed.info())
describe_fdic = fdic_failed.describe(include='all')
describe_fdic


In [ ]:
# Exploración de valores nulos en columnas FDIC
nulls_fdic = fdic_failed.isnull().sum().reset_index(name='nulls')
nulls_fdic.rename(columns={'index': 'column'}, inplace=True)
nulls_fdic['pct'] = nulls_fdic['nulls'] / len(fdic_failed) * 100
nulls_fdic


In [ ]:
# Visualizar comportamiento temporal de las quiebras
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

failures_year = fdic_failed.groupby('year').size()
axes[0, 0].bar(failures_year.index, failures_year.values, color='crimson', alpha=0.8)
axes[0, 0].set_title('Quiebras por Año')
axes[0, 0].set_xlabel('Año')
axes[0, 0].set_ylabel('Conteo')
axes[0, 0].grid(True, alpha=0.3)

# Evolución acumulada para dimensionar periodos críticos
cumulative = failures_year.cumsum()
axes[0, 1].plot(cumulative.index, cumulative.values, marker='o', color='darkred')
axes[0, 1].fill_between(cumulative.index, cumulative.values, alpha=0.3, color='lightcoral')
axes[0, 1].set_title('Quiebras acumuladas')
axes[0, 1].set_xlabel('Año')
axes[0, 1].set_ylabel('Acumulado')
axes[0, 1].grid(True, alpha=0.3)

# Distribución intra-anual por trimestre
if 'quarter' in fdic_failed:
    failures_quarter = fdic_failed['quarter'].value_counts().sort_index()
    axes[1, 0].bar(failures_quarter.index, failures_quarter.values, color='tomato', alpha=0.8)
    axes[1, 0].set_title('Quiebras por Trimestre')
    axes[1, 0].set_xlabel('Trimestre')
    axes[1, 0].set_ylabel('Conteo')
    axes[1, 0].grid(True, alpha=0.3)

# Ranking de los diez años con más cierres bancarios
failures_top_years = failures_year.nlargest(10)
axes[1, 1].barh(failures_top_years.index.astype(str), failures_top_years.values, color='firebrick', alpha=0.8)
axes[1, 1].invert_yaxis()
axes[1, 1].set_title('Top 10 Años con Más Quiebras')
axes[1, 1].set_xlabel('Conteo')
axes[1, 1].grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Analizar concentración geográfica de quiebras
state_map = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
    'DC': 'District of Columbia', 'PR': 'Puerto Rico'
}

fdic_failed['State_Name'] = fdic_failed['State'].map(state_map).fillna(fdic_failed['State'])
state_counts = fdic_failed['State_Name'].value_counts()

plt.figure(figsize=(14, 8))
state_counts.head(20).iloc[::-1].plot(kind='barh', color='teal', alpha=0.8)
plt.title('Top 20 Estados con Mayor Número de Quiebras')
plt.xlabel('Número de quiebras')
plt.ylabel('Estado')
plt.grid(True, axis='x', alpha=0.3)
plt.show()

state_counts.head(20)


## 3. Datos FFIEC: carga y limpieza

Verificamos la carpeta de schedules FFIEC, implementamos una utilidad para leer archivos delimitados por tabuladores y cargamos los schedules principales:
- **RC** para obtener el balance general.
- **ENT** con información institucional.
- **POR** que incluye el `Certificate Number` necesario para enlazar con FDIC.


In [ ]:
# Revisar disponibilidad de archivos FFIEC en el directorio esperado
ffiec_dir = Path('FFIEC CDR Call Bulk All Schedules 03312001')
if not ffiec_dir.exists():
    raise FileNotFoundError('No se encontró la carpeta con los schedules FFIEC.')

print(f'Se detectaron {len(list(ffiec_dir.glob("*.txt")))} archivos FFIEC en {ffiec_dir}.')


In [ ]:
# Utilidad para cargar schedules FFIEC tomando en cuenta encabezados y filas descriptivas
def load_ffiec_schedule(filename, columns=None, drop_description=True):
    filepath = ffiec_dir / filename
    with filepath.open(encoding='utf-8', newline='') as f:
        reader = csv.reader(f, delimiter='	')
        header = [h.strip('"').strip() for h in next(reader)]
        first_row = next(reader, [])
        if drop_description and (not first_row or not first_row[0].strip()):
            iterator = reader
        else:
            iterator = itertools.chain([first_row], reader)

        if columns:
            cols = []
            for col in columns:
                name = col.strip('"').strip()
                if name in header and name not in cols:
                    cols.append(name)
            if 'IDRSSD' not in cols and 'IDRSSD' in header:
                cols.insert(0, 'IDRSSD')
            indices = {col: header.index(col) for col in cols}
            data = {col: [] for col in cols}
            for row in iterator:
                if not any(row):
                    continue
                for col, idx in indices.items():
                    value = row[idx] if idx < len(row) else ''
                    data[col].append(value.strip())
            df = pd.DataFrame(data)
        else:
            records = [row for row in iterator if any(row)]
            df = pd.DataFrame(records, columns=header)

    df = df.loc[:, [c for c in df.columns if c]]
    if 'IDRSSD' in df.columns:
        df['IDRSSD'] = df['IDRSSD'].astype(str).str.strip()
    return df


In [ ]:
# Identificar archivos específicos a utilizar en el análisis
rc_file = next((f.name for f in ffiec_dir.glob('*Schedule RC*03312001.txt')), None)
ent_file = next((f.name for f in ffiec_dir.glob('*Schedule ENT*.txt')), None)
por_file = next((f.name for f in ffiec_dir.glob('FFIEC CDR Call Bulk POR*.txt')), None)

print('Schedule RC:', rc_file)
print('Schedule ENT:', ent_file)
print('Schedule POR:', por_file)


In [ ]:
# Cargar columnas clave del balance general con respaldo RCFD/RCON
balance_columns = [
    'IDRSSD',
    'RCFD2170', 'RCON2170',
    'RCFD2200', 'RCON2200',
    'RCFD3210', 'RCON3210',
    'RCFD2948', 'RCON2948',
    'RCFD3123', 'RCON3123',
    'RCFD3190', 'RCON3190'
]

df_rc = load_ffiec_schedule(rc_file, columns=balance_columns) if rc_file else None
print(df_rc.head())
print(df_rc.shape if df_rc is not None else None)


In [ ]:
# Cargar metadatos de entidad (ENT) y certificados (POR)
df_ent = load_ffiec_schedule(ent_file, drop_description=False) if ent_file else None
if df_ent is not None:
    df_ent = df_ent.loc[:, ~df_ent.columns.str.fullmatch('')]
print(df_ent.head() if df_ent is not None else None)
print(df_ent.shape if df_ent is not None else None)

df_por = load_ffiec_schedule(por_file, drop_description=False) if por_file else None
if df_por is not None:
    df_por = df_por.rename(columns={'FDIC Certificate Number': 'Cert'})
    df_por['IDRSSD'] = df_por['IDRSSD'].astype(str).str.strip()
    df_por['Cert'] = df_por['Cert'].astype(str).str.strip()
print(df_por.head() if df_por is not None else None)
print(df_por.shape if df_por is not None else None)


## 4. Variables clave y ratios financieros

Mapeamos los códigos de columnas relevantes (activos, depósitos, capital, etc.) buscando tanto la versión RCFD como RCON. A partir de esos totales calculamos ratios de solvencia, liquidez y calidad de cartera, además de revisar la cobertura de cada métrica.


In [ ]:
# Inventario de variables financieras disponibles en el Schedule RC
balance_map = {
    'total_assets': {'label': 'Total Assets', 'codes': ['RCFD2170', 'RCON2170']},
    'total_deposits': {'label': 'Total Deposits', 'codes': ['RCFD2200', 'RCON2200']},
    'total_equity': {'label': 'Total Equity Capital', 'codes': ['RCFD3210', 'RCON3210']},
    'loans_net': {'label': 'Loans & Leases Net', 'codes': ['RCFD2948', 'RCON2948']},
    'allowance': {'label': 'Allowance for Loan & Lease Losses', 'codes': ['RCFD3123', 'RCON3123']},
    'total_liabilities': {'label': 'Total Liabilities', 'codes': ['RCFD3190', 'RCON3190']},
}

if df_rc is not None:
    for alias, meta in balance_map.items():
        codes_available = [code for code in meta['codes'] if code in df_rc.columns]
        if codes_available:
            print(f"{meta['label']}: {codes_available}")
        else:
            print(f"{meta['label']}: no disponible en este corte")


In [ ]:
# Funciones de apoyo para seleccionar columnas numéricas y calcular ratios
def select_numeric(df, codes):
    for code in codes:
        if code in df.columns:
            series = pd.to_numeric(df[code], errors='coerce')
            if series.notna().any():
                return series
    return pd.Series(np.nan, index=df.index)

def safe_ratio(num, den):
    ratio = num / den
    return ratio.where(den.notna() & (den != 0))

if df_rc is not None:
    df_rc_ratios = pd.DataFrame({'IDRSSD': df_rc['IDRSSD']})
    for alias, meta in balance_map.items():
        df_rc_ratios[alias] = select_numeric(df_rc, meta['codes'])

    # Ratios de solvencia, liquidez y calidad de cartera
    df_rc_ratios['equity_to_assets'] = safe_ratio(df_rc_ratios['total_equity'], df_rc_ratios['total_assets'])
    df_rc_ratios['loan_to_deposit'] = safe_ratio(df_rc_ratios['loans_net'], df_rc_ratios['total_deposits'])
    df_rc_ratios['allowance_to_loans'] = safe_ratio(df_rc_ratios['allowance'], df_rc_ratios['loans_net'])
    df_rc_ratios['deposits_to_assets'] = safe_ratio(df_rc_ratios['total_deposits'], df_rc_ratios['total_assets'])
    df_rc_ratios['debt_to_equity'] = safe_ratio(df_rc_ratios['total_liabilities'], df_rc_ratios['total_equity'])

    ratio_cols = ['equity_to_assets', 'loan_to_deposit', 'allowance_to_loans', 'deposits_to_assets', 'debt_to_equity']
    coverage = (df_rc_ratios[ratio_cols].notna().sum() / len(df_rc_ratios) * 100).round(1)
    print('Cobertura de ratios (% instituciones con valor):')
    print(coverage)

    df_rc_ratios[ratio_cols].describe()


Se escogieron ratios básicos de solvencia y liquidez (equity_to_assets, loan_to_deposit, allowance_to_loans, deposits_to_assets, debt_to_equity)

In [ ]:
# Distribución de ratios para detectar sesgos y valores extremos
if df_rc is not None:
    ratio_cols = ['equity_to_assets', 'loan_to_deposit', 'allowance_to_loans', 'deposits_to_assets', 'debt_to_equity']
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    axes = axes.flatten()
    for idx, col in enumerate(ratio_cols):
        data = df_rc_ratios[col].dropna()
        if data.empty:
            axes[idx].set_visible(False)
            continue
        q1, q99 = data.quantile([0.01, 0.99])
        data = data[(data >= q1) & (data <= q99)]
        axes[idx].hist(data, bins=40, color='steelblue', alpha=0.8, edgecolor='black')
        axes[idx].axvline(data.mean(), color='orange', linestyle='--', lw=2, label=f'Media: {data.mean():.3f}')
        axes[idx].axvline(data.median(), color='red', linestyle='--', lw=2, label=f'Mediana: {data.median():.3f}')
        axes[idx].set_title(col.replace('_', ' ').title())
        axes[idx].grid(True, alpha=0.3)
        axes[idx].legend()
    for idx in range(len(ratio_cols), len(axes)):
        axes[idx].set_visible(False)
    plt.tight_layout()
    plt.show()


## 5. Integración FDIC + FFIEC

Unimos los datasets del FFIEC (ENT + ratios) utilizando `IDRSSD` y anexamos el certificado desde POR. Con el `Cert` homogenizado cruzamos contra la lista de la FDIC para generar la etiqueta binaria `FAILED` que indica si la institución quebró.


In [ ]:
# Construir marco con certificados para enlazar FDIC-FFIEC
if df_por is not None:
    cert_map = df_por[['IDRSSD', 'Cert']].copy()
else:
    cert_map = pd.DataFrame(columns=['IDRSSD', 'Cert'])
print(f'Certificados disponibles: {cert_map["Cert"].notna().sum():,} / {len(cert_map):,}')


In [ ]:
# Unificar ENT + ratios y anexar certificados
ffiec = df_ent.copy()
ffiec['IDRSSD'] = ffiec['IDRSSD'].astype(str).str.strip()
ffiec = ffiec.merge(df_rc_ratios, on='IDRSSD', how='inner')
ffiec = ffiec.merge(cert_map, on='IDRSSD', how='left')
ffiec['Cert'] = ffiec['Cert'].astype(str).str.strip()
print(f'Dataset FFIEC unificado: {ffiec.shape}')
ffiec.head()


In [ ]:
# Etiquetar quiebra (FAILED) usando el certificado cruzado
ffiec['FAILED'] = ffiec['Cert'].isin(fdic_failed['Cert']).astype(int)
failed_counts = ffiec['FAILED'].value_counts()
print(failed_counts)
print(f"Proporción de quiebras: {ffiec['FAILED'].mean()*100:.2f}%")


## 6. Comparativa de indicadores

Analizamos cómo se comportan los ratios entre bancos quebrados y solventes. Primero calculamos estadísticas comparativas y luego trazamos histogramas y boxplots filtrando valores extremos para destacar diferencias reales entre grupos.


In [ ]:
# Estadísticas comparativas entre bancos quebrados y solventes
ratios = ['equity_to_assets', 'loan_to_deposit', 'allowance_to_loans', 'deposits_to_assets', 'debt_to_equity']
comparison_stats = []
for col in ratios:
    failed = ffiec.loc[ffiec['FAILED'] == 1, col].dropna()
    solvent = ffiec.loc[ffiec['FAILED'] == 0, col].dropna()
    if failed.empty or solvent.empty:
        continue
    comparison_stats.append({
        'ratio': col,
        'failed_mean': failed.mean(),
        'solvent_mean': solvent.mean(),
        'failed_median': failed.median(),
        'solvent_median': solvent.median(),
        'pct_diff_mean': (failed.mean() - solvent.mean()) / solvent.mean() * 100,
        'failed_n': len(failed),
        'solvent_n': len(solvent)
    })

pd.DataFrame(comparison_stats)


In [ ]:
# Gráficas comparativas para visualizar diferencias de distribución
fig, axes = plt.subplots(len(ratios), 2, figsize=(18, 6 * len(ratios)))
for idx, col in enumerate(ratios):
    failed = ffiec.loc[ffiec['FAILED'] == 1, col].dropna()
    solvent = ffiec.loc[ffiec['FAILED'] == 0, col].dropna()
    if failed.empty or solvent.empty:
        axes[idx, 0].set_visible(False)
        axes[idx, 1].set_visible(False)
        continue
    q_low_f, q_high_f = failed.quantile([0.01, 0.99])
    q_low_s, q_high_s = solvent.quantile([0.01, 0.99])
    failed_f = failed[(failed >= q_low_f) & (failed <= q_high_f)]
    solvent_f = solvent[(solvent >= q_low_s) & (solvent <= q_high_s)]

    axes[idx, 0].hist(solvent_f, bins=40, alpha=0.6, label=f'Solventes (n={len(solvent_f)})', color='green', edgecolor='black')
    axes[idx, 0].hist(failed_f, bins=40, alpha=0.6, label=f'Quebrados (n={len(failed_f)})', color='red', edgecolor='black')
    axes[idx, 0].set_title(f'Distribución de {col}')
    axes[idx, 0].legend()
    axes[idx, 0].grid(True, alpha=0.3)

    axes[idx, 1].boxplot([solvent_f, failed_f], labels=['Solventes', 'Quebrados'], widths=0.6, patch_artist=True)
    axes[idx, 1].set_title(f'Comparación de {col}')
    axes[idx, 1].grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


## 7. Correlaciones y matriz de calor

Calculamos la correlación de cada ratio con la variable objetivo `FAILED` y visualizamos la matriz completa. Esto ayuda a identificar qué indicadores están más asociados con la quiebra y posibles redundancias entre variables.


In [ ]:
# Calcular correlaciones de los ratios con la etiqueta de quiebra
subset = ffiec[ratios + ['FAILED']].dropna()
corr_matrix = subset.corr()
print('Correlaciones con FAILED:')
print(corr_matrix['FAILED'].sort_values(ascending=False))


In [ ]:
# Visualizar correlaciones completas en una matriz de calor
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Matriz de Correlación (Ratios + FAILED)')
plt.show()


## 8. Valores faltantes y calidad de datos

Evaluamos cuántos valores faltantes quedan en el dataset integrado. El resumen numérico y la gráfica permiten priorizar columnas para imputación o descarte antes de entrenar modelos predictivos.


In [ ]:
# Inventario de valores faltantes tras la integración FFIEC
missing_summary = ffiec.isnull().sum().reset_index(name='nulls')
missing_summary.rename(columns={'index': 'column'}, inplace=True)
missing_summary['pct'] = missing_summary['nulls'] / len(ffiec) * 100
missing_summary.sort_values('pct', ascending=False).head(20)


In [ ]:
# Visualizar columnas con mayor porcentaje de valores nulos
plt.figure(figsize=(14, 8))
top_missing = missing_summary.sort_values('pct', ascending=False).head(15)
plt.barh(top_missing['column'], top_missing['pct'], color='coral', alpha=0.8)
plt.gca().invert_yaxis()
plt.title('Top 15 Columnas con Mayor Porcentaje de Valores Faltantes')
plt.xlabel('% de valores faltantes')
plt.grid(True, axis='x', alpha=0.3)
plt.show()


## 9. Dataset final preparado

Seleccionamos las columnas relevantes para modelado, eliminamos registros sin ratios calculados y guardamos un archivo limpio (`banking_dataset_processed_v2.csv`) listo para usarse en clasificación o análisis de supervivencia.


In [ ]:
# Armar dataset depurado con variables relevantes para modelado
feature_cols = ratios + ['total_assets', 'total_deposits', 'total_equity', 'loans_net', 'allowance', 'total_liabilities', 'Cert']
feature_cols = [col for col in feature_cols if col in ffiec.columns]

model_df = ffiec[feature_cols + ['FAILED']].copy()
model_df_clean = model_df.dropna(subset=ratios)
print(f'Dataset para modelado: {model_df_clean.shape}')
model_df_clean.head()


In [ ]:
# Guardar dataset listo para experimentos de modelado
output_path = Path('banking_dataset_processed_v2.csv')
model_df_clean.to_csv(output_path, index=False)
print(f'Dataset guardado en {output_path.resolve()}')


## 10. Conclusiones y siguientes pasos

Cerramos con un resumen de hallazgos y un checklist de tareas futuras: ampliar variables (features temporales), atender el desbalance de clases y preparar un esquema de validación temporal antes de probar modelos.
